In [1]:
########### THIS CODE WAS USED TO CREATE A RESULT CSV FILE FROM THE JSON FILE(total 8 json predicted files got generate from batch prediction which you can see in 'skin_cancer_project2/prediction/prediction-Skin_Cancer-2023-11-02T04:59:31.158832Z') OF BATCH PREDICTION FROM THE BUCKET skin_cancer_project2
########## RESULT FILE WAS SAVED AS results.csv


# Import required libraries
import ndjson
import pandas as pd
from google.cloud import aiplatform
from google.cloud import storage

# Initialize Vertex AI connection
aiplatform.init(location='us-central1')

# Define Cloud Storage client
client = storage.Client()

ModuleNotFoundError: No module named 'ndjson'

In [ ]:
pip install ndjson


In [17]:
# Initialize Vertex AI connection
aiplatform.init(location='us-central1')

# Define Cloud Storage client
client = storage.Client()

In [11]:
pip install google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


In [14]:
import aiplatform

ModuleNotFoundError: No module named 'aiplatform'

In [19]:
# Initialize Vertex AI connection
#aiplatform.init(location='us-central1')
from google.cloud import storage
# Define Cloud Storage client
client = storage.Client()

In [21]:

# Get the batch prediction job
job_id = '3235574442339860480' #E.g. 4897355631998009322


In [22]:
batch_prediction_job = aiplatform.BatchPredictionJob(job_id)

In [23]:
# Get the job output folder
results_folder = batch_prediction_job.output_info.gcs_output_directory
results_folder

'gs://skin_cancer_project2/prediction/prediction-Skin_Cancer-2023-11-02T04:59:31.158832Z'

In [24]:
# Create prefix to filter blobs
prefix_filter = results_folder.replace('gs://skin_cancer_project2/','')
prefix_filter

'prediction/prediction-Skin_Cancer-2023-11-02T04:59:31.158832Z'

In [25]:
# Define Cloud Storage client and bucket where the results are stored
client = storage.Client()
bucket = client.get_bucket('skin_cancer_project2')

In [26]:
# Create iterator
blobs = client.list_blobs('skin_cancer_project2', prefix=prefix_filter)

In [27]:
# Show all files with predictions
for blob in blobs:
    print(blob.name)

prediction/prediction-Skin_Cancer-2023-11-02T04:59:31.158832Z/predictions_00001.jsonl
prediction/prediction-Skin_Cancer-2023-11-02T04:59:31.158832Z/predictions_00002.jsonl
prediction/prediction-Skin_Cancer-2023-11-02T04:59:31.158832Z/predictions_00003.jsonl
prediction/prediction-Skin_Cancer-2023-11-02T04:59:31.158832Z/predictions_00004.jsonl
prediction/prediction-Skin_Cancer-2023-11-02T04:59:31.158832Z/predictions_00005.jsonl
prediction/prediction-Skin_Cancer-2023-11-02T04:59:31.158832Z/predictions_00006.jsonl
prediction/prediction-Skin_Cancer-2023-11-02T04:59:31.158832Z/predictions_00007.jsonl
prediction/prediction-Skin_Cancer-2023-11-02T04:59:31.158832Z/predictions_00008.jsonl
prediction/prediction-Skin_Cancer-2023-11-02T04:59:31.158832Z/predictions_00009.jsonl
prediction/prediction-Skin_Cancer-2023-11-02T04:59:31.158832Z/predictions_00010.jsonl


In [28]:
blobs = client.list_blobs('skin_cancer_project2', prefix=prefix_filter)

# Display few predictions to show prediction format
for blob in blobs:
    # Load data and transform into json
    json_data_string = blob.download_as_string()
    json_data = ndjson.loads(json_data_string)
    pred_examples = json_data[0:3]
    break
    
pred_examples

[{'instance': {'content': 'gs://skin_cancer_project2/Test_Images/ISIC_0035321.jpg'},
  'prediction': {'ids': ['7852593573198299136',
    '121320387886710784',
    '6277459603525468160',
    '8235681016501501952',
    '5842017814554083328',
    '7548882073327501312',
    '6994939319160930304'],
   'displayNames': ['bcc_images',
    'nv_images',
    'df_images',
    'akiec_images',
    'mel_images',
    'vasc_images',
    'bkl_images'],
   'confidences': [0.9025179,
    0.053746846,
    0.014281384,
    0.009276812,
    0.007934716,
    0.006167055,
    0.006075252]}},
 {'instance': {'content': 'gs://skin_cancer_project2/Test_Images/ISIC_0035857.jpg'},
  'prediction': {'ids': ['121320387886710784',
    '6277459603525468160',
    '7852593573198299136',
    '8235681016501501952',
    '5842017814554083328',
    '7548882073327501312',
    '6994939319160930304'],
   'displayNames': ['nv_images',
    'df_images',
    'bcc_images',
    'akiec_images',
    'mel_images',
    'vasc_images',
    'b

In [29]:
blobs = client.list_blobs('skin_cancer_project2', prefix=prefix_filter)

# Create placeholders for prediction results
cs_uri_list = []
predicted_label_list = []
prediction_confidence_list = []

# Iterate over each file in Cloud Storage
for blob in blobs:
    # Load data and transform into json
    json_data_string = blob.download_as_string()
    json_data = ndjson.loads(json_data_string) 
    
    # Iterate over each prediction in file
    for prediction in json_data:
        # Get data from prediction
        cs_uri = prediction.get('instance').get('content')
        predicted_label = prediction.get('prediction').get('displayNames')[0] # This array is sorted so the highest probability label is always on 1st position.
        prediction_confidence = prediction.get('prediction').get('confidences')[0]
        
        # Append to appropriate list
        cs_uri_list.append(cs_uri)
        predicted_label_list.append(predicted_label)
        prediction_confidence_list.append(prediction_confidence)

In [30]:
# Create data frame with results. 
results_df = pd.DataFrame({'cs_uri': cs_uri_list, 'predicted_label': predicted_label_list, 'confidence': prediction_confidence_list})
results_df.head()

,cs_uri,predicted_label,confidence
0,gs://skin_cancer_project2/Test_Images/ISIC_003...,bcc_images,0.902518
1,gs://skin_cancer_project2/Test_Images/ISIC_003...,nv_images,0.809563
2,gs://skin_cancer_project2/Test_Images/ISIC_003...,nv_images,0.719921
3,gs://skin_cancer_project2/Test_Images/ISIC_003...,bkl_images,0.868703
4,gs://skin_cancer_project2/Test_Images/ISIC_003...,bcc_images,0.914819


In [31]:
# Define the CSV file path on Cloud Storage
csv_file_path = 'gs://skin_cancer_project2/results.csv'  # Specify the desired file name and path


In [32]:
# Save the DataFrame as a CSV file
results_df.to_csv(csv_file_path, index=False)


In [ ]:
# Upload the CSV file to the Google Cloud Storage bucket
blob = bucket.blob('results.csv')
blob.upload_from_filename(csv_file_path)

print(f"CSV file saved and uploaded to {csv_file_path}")
